# Process data config

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('..')

from os import environ
import numpy as np
import pandas as pd
from indicators import indicators
from datetime import timedelta
from tqdm.auto import tqdm
from config.config import ConfigFactory

import lightgbm as lgb

from eli5.sklearn import PermutationImportance
from shaphypetune import BoostBoruta

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss, mean_squared_error, accuracy_score, precision_score

import torch
from torch.utils.data import TensorDataset

from colorama import Style, Fore


class CFG:
    collect_data = True # create new dataset or load previous
    select_features = True
    train_NN = False
    train_LGBM = True
    n_repeats = 1
    n_folds = 5
    cls_target_ratio = 1.021

/home/alex/.local/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/alex/.local/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_

# Load data and add indicators

In [2]:
# Set environment variable
environ["ENV"] = "1h_4h"

# Get configs
configs = ConfigFactory.factory(environ).configs

def get_file(ticker):
    ''' Find files buy ticker names, file names can be in different formats '''
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    return None, None

def add_indicators(df, ttype, configs):
    # add RSI
    rsi = indicators.RSI(ttype, configs)
    df = rsi.get_indicator(df, '', '', 0)
    # add RSI
    stoch = indicators.STOCH(ttype, configs)
    df = stoch.get_indicator(df, '', '', 0)
    # add Trend
    trend = indicators.Trend(ttype, configs)
    df = trend.get_indicator(df, '', '', 0)
    # add MACD
    macd = indicators.MACD(ttype, configs)
    df = macd.get_indicator(df, '', '', 0)
    # add ATR
    atr = indicators.ATR(ttype, configs)
    df = atr.get_indicator(df, '', '', 0)
    # add SMA
    # sma = indicators.SMA(ttype, configs)
    # df = sma.get_indicator(df, '', '', 0)
    return df

def create_train_df(df, ttype, configs, target_offset, first, last, step):
    ''' Create train dataset from signal statistics and ticker candle data'''
    train_df = pd.DataFrame()
    tickers = df['ticker'].unique()
    
    for ticker in tqdm(tickers):
        # get signals with current ticker
        signal_df = df[df['ticker'] == ticker]
        times = signal_df['time']
        
        # load candle history of this ticker
        tmp_df_1h, tmp_df_4h = get_file(ticker)

        # add indicators 
        tmp_df_1h = add_indicators(tmp_df_1h, ttype, configs)

        # add historical data for current ticker
        for i, t in enumerate(times.to_list()):
            pass_cycle = False
            pattern = signal_df.iloc[i, signal_df.columns.get_loc('pattern')]
            row = tmp_df_1h.loc[tmp_df_1h['time'] == t, :].reset_index(drop=True)
            
            for i in range(first, last + step, step):
                time_prev = t + timedelta(hours= -i)
                try:
                    row_tmp = tmp_df_1h.loc[tmp_df_1h['time'] == time_prev, :].reset_index(drop=True)
                    row_tmp.columns = [c + f'_prev_{i}' for c in row_tmp.columns]
                except IndexError:
                    pass_cycle = True
                    break
                row = pd.concat([row, row_tmp.iloc[:,1:]], axis=1)
                row['ticker'] = ticker
                row['pattern'] = pattern
                
            if pass_cycle:
                continue

            row['target'] = 0
            
            if row['pattern'].values == 'STOCH_RSI':
                if ttype == 'buy':
                    row['ttype'] = 'sell'
                else:
                    row['ttype'] = 'buy'
            else:
                row['ttype'] = ttype
            
            # Ff ttype = buy and during the selected period high price was higher than close_price * target_ratio
            # and earlier low price wasn't lower than close_price / target_ratio, than target is True, else target is False.
            # Similarly for ttype = sell 
            close_price = tmp_df_1h.loc[tmp_df_1h['time'] == t, 'close'].values
            
            for i in range(1, target_offset + 1):
                time_next = t + timedelta(hours=i)
                target_buy = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'high'].reset_index(drop=True)
                target_sell = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'low'].reset_index(drop=True)

                try:
                    target_buy = target_buy > close_price * CFG.cls_target_ratio
                    target_sell = target_sell < close_price / CFG.cls_target_ratio
                except ValueError:
                    pass_cycle = True
                    break
                
                try:
                    if (row['ttype'].values == 'buy' and target_sell[0]) or (row['ttype'].values == 'sell' and target_buy[0]):
                        break
                    elif (row['ttype'].values == 'buy' and target_buy[0]) or (row['ttype'].values == 'sell' and target_sell[0]):
                        row['target'] = 1
                        break
                except KeyError:
                    pass_cycle = True
                    break
            
            if pass_cycle:
                continue

            # add data to the dataset
            if train_df.shape[0] == 0:
                train_df = row
            else:
                train_df = pd.concat([train_df, row])
    
    return train_df

# for how long time (in hours) we want to predict
target_offset = 24
# first previous data point to collect for model training (value represents number of hours before signal point)
first = 4
# last previous data point to collect for model training (value represents number of hours before signal point)
last = 72
# step of previous data points collecting (total number of points to collect is (last - first + step) / step)
step = 4

if CFG.collect_data is True:
    # Buy
    # dataset with the signal statistics
    df = pd.read_pickle('signal_stat/buy_stat_1h.pkl')
    # dataset for model train
    train_buy = create_train_df(df, 'buy', configs, target_offset, first, last, step)
    train_buy = train_buy.dropna()

    # Sell
    # dataset with the signal statistics
    df = pd.read_pickle('signal_stat/sell_stat_1h.pkl')
    # dataset for model train
    train_sell = create_train_df(df, 'sell', configs, target_offset, first, last, step)
    train_sell = train_sell.dropna()

    train_df = pd.concat([train_buy, train_sell]).sort_values('time').reset_index(drop=True)
    train_df.to_pickle(f'signal_stat/train_df_{last}.pkl')
else:
    train_df = pd.read_pickle(f'signal_stat/train_df_{last}.pkl')

display(train_df.head())
display(train_df.shape)


  0%|          | 0/352 [00:00<?, ?it/s]

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty

  0%|          | 0/352 [00:00<?, ?it/s]

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty

,time,open,high,low,close,volume,rsi,stoch_slowk,stoch_slowd,stoch_slowk_dir,...,linear_reg_angle_prev_72,macd_prev_72,macdsignal_prev_72,macdhist_prev_72,macd_dir_prev_72,macdsignal_dir_prev_72,atr_prev_72,close_smooth_prev_72,target,ttype
0,2022-09-10 21:00:00,0.9999,0.9999,0.9998,0.9998,227818.00,58.380200,35.714286,34.523810,0.205808,...,-0.856226,-0.000020,-0.000010,-0.000009,0.318808,0.227337,0.000114,1.000004,0,buy
1,2022-09-15 15:00:00,1.0000,1.0000,0.9999,1.0000,250135.00,58.463239,64.285714,53.174603,0.357436,...,5.250146,0.000016,0.000011,0.000006,0.229362,0.110533,0.000113,0.999742,0,buy
2,2022-09-21 19:00:00,1.0000,1.0000,0.9999,0.9999,49801.00,45.480088,57.142857,61.904762,-0.066667,...,5.078633,0.000006,-0.000004,0.000010,-1.898082,-0.192558,0.000113,0.999988,0,buy
3,2022-12-25 15:00:00,0.3970,0.3980,0.3930,0.3960,304916.80,36.897427,42.857143,43.650794,-0.069109,...,0.303562,-0.000151,-0.000647,0.000496,-0.292488,-0.109834,0.003108,0.398667,0,sell
4,2022-12-25 15:00:00,6.8900,6.9100,6.8900,6.9000,3986.13,30.533005,14.404762,18.392857,-0.204445,...,11.812509,0.028818,-0.003283,0.032101,-0.946146,-0.453481,0.088674,6.850417,0,sell


(11601, 385)

# Check target correctness

In [3]:
# i = 1000

# x = train_df.loc[(train_df.target == 1) & (train_df.ttype == 'buy'), ['ticker', 'ttype', 'pattern', 'time', 'close', 'target']]
# y = x.iloc[i]
# low_price, high_price = y['close'] / CFG.cls_target_ratio, y['close'] * CFG.cls_target_ratio,
# print(y['ticker'], y['time'], y['ttype'])

# tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"]}_1h.pkl')
# # tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"][:-4]}-{y["ticker"][-4:]}_1h.pkl')
# # tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{y["ticker"][:-4]}-{y["ticker"][-4:]}-SWAP_4h.pkl')

# tmp_df_1h['low_price'] = low_price
# tmp_df_1h['high_price'] = high_price
# idx = tmp_df_1h[tmp_df_1h['time'] == y['time']].index[0]

# tmp_df_1h = tmp_df_1h.iloc[idx:idx+24][['time', 'close', 'high', 'high_price', 'low', 'low_price']]

# if y['ttype'] == 'buy':
#     tmp_df_1h['signal'] = tmp_df_1h['high'] > tmp_df_1h['high_price']
#     tmp_df_1h['anti_signal'] = tmp_df_1h['low'] < tmp_df_1h['low_price']
# else:
#     tmp_df_1h['signal'] = tmp_df_1h['low'] < tmp_df_1h['low_price']
#     tmp_df_1h['anti_signal'] = tmp_df_1h['high'] > tmp_df_1h['high_price']

# tmp_df_1h

# Pytorch

### Create dataset

In [4]:
test_size=0.2

x_data = train_df.drop(['target', 'time', 'ticker', 'pattern', 'ttype'], axis=1)
y_data = train_df['target'] >= train_df['close']
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=test_size, shuffle=True)

scaler = StandardScaler()
x_train[x_train.columns] = scaler.fit_transform(x_train)
x_valid[x_valid.columns] = scaler.transform(x_valid)

x_train = torch.tensor(x_train.values, dtype=torch.float32)
x_valid = torch.tensor(x_valid.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_valid = torch.tensor(y_valid.values, dtype=torch.float32)

display(type(x_train), type(y_train))

torch.Tensor

torch.Tensor

### Find available device

In [5]:
# find available device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Build model

In [6]:
class SigModel(torch.nn.Module):
    def __init__(self):
        super(SigModel, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('lin1', torch.nn.Linear(260, 64))
        self.layers.add_module('relu1', torch.nn.ReLU())
        self.layers.add_module('do1', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin2', torch.nn.Linear(64, 128))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('do2', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin3', torch.nn.Linear(128, 96))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('do3', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin4', torch.nn.Linear(96, 32))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('do4', torch.nn.Dropout(p=0.25))
        self.layers.add_module('lin5', torch.nn.Linear(32, 1))
        self.layers.add_module('sigmoid', torch.nn.Sigmoid())
    
    def forward(self, input):
        return self.layers(input)

# Train model

In [7]:
from torch import nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

# train function
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    # put the model in train mode
    model.train()
    
    x_train, x_valid, y_train, y_valid = train_loader

    # get output of the model
    train_preds = model(x_train).squeeze()
    # calculate train loss
    train_loss = criterion(train_preds, y_train)
    train_acc = (train_preds.round() == y_train).float().mean()
    
    # set gradient to zero to prevent it accumulation
    optimizer.zero_grad() # ~ model.zero_grad()
    # calculate gradient
    train_loss.backward() 
    # update weights
    optimizer.step()
    
    # put the model in evaluation mode
    model.eval()

    with torch.no_grad():
        val_preds = model(x_valid).squeeze()
        val_loss = criterion(val_preds, y_valid)
        val_acc = (val_preds.round() == y_valid).float().mean()
    
    # update weights according to gradient value
    scheduler.step(val_loss)
    
    return train_loss, train_acc, val_loss, val_acc

# Initialize model
if CFG.train_NN:
    model = SigModel().to(device)

    # Number of epochs
    epochs = 10000

    # Send data to the device
    x_train, x_valid = x_train.to(device), x_valid.to(device)
    y_train, y_valid = y_train.to(device), y_valid.to(device)
    train_loader = x_train, x_valid, y_train, y_valid

    # Empty loss lists to track values
    epoch_count, train_loss_values, valid_loss_values = [], [], []

    criterion = nn.BCELoss()
    learning_rate = 1e-6
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2000, threshold=1e-2)

    # Loop through the data
    for epoch in range(epochs):
        train_loss, train_acc, val_loss, val_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler)

        # Print progress a total of 20 times
        if epoch % int(epochs / 20) == 0:
            print(f'Epoch: {epoch:4.0f} | Train Loss: {train_loss:.5f}, Train Acc: {train_acc:.5f}\
                Validation Loss: {val_loss:.5f}, Val Acc: {val_acc:.5f}\
                    LR: {optimizer.state_dict()["param_groups"][0]["lr"]}')

            epoch_count.append(epoch)
            train_loss_values.append(train_loss.cpu().detach().numpy())
            valid_loss_values.append(val_loss.cpu().detach().numpy())


# Plot NN train results

In [8]:
import matplotlib.pyplot as plt

if CFG.train_NN:
    plt.plot(epoch_count, train_loss_values, label='Training Loss')
    plt.plot(epoch_count, valid_loss_values, label='Validation Loss')
    plt.title('Training & Validation Loss Curves')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

# Select features

In [10]:
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

def lgbm_tuning(df, permut=False, boruta=False):
    features = [c for c in df.columns if c not in ['time', 'target', 'ticker', 'pattern', 'ttype']]
    groups = df['ticker']

    outer_cv_score = [] # store all cv scores of outer loop inference

    perm_df_ = pd.DataFrame()
    feature_importances_ = pd.DataFrame()
    boruta_df_ = pd.DataFrame()
    
    for i in range(CFG.n_repeats):
        print(f'Repeat {blu}#{i+1}')
        
        if task_type == 'cls':
            y_fold = df['target'] >= df['close']
            kf = StratifiedGroupKFold(n_splits=CFG.n_folds, shuffle=True, random_state=180820231)
            eval_metric = 'logloss'
        else:
            y_fold = (df['target'] - df['close']) / df['close']
            kf = GroupKFold(n_splits=CFG.n_folds)
            eval_metric = 'mse'

        X, y = df[features], y_fold
        oof = np.zeros(len(df))
        models_ = [] # Used to store models trained in the inner loop.
        
        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X, y, groups)):
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            if task_type == 'cls':
                clf = lgb.LGBMClassifier(**params)
            else:
                clf = lgb.LGBMRegressor(**params)
            clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                    eval_metric=eval_metric, 
                    callbacks=[lgb.log_evaluation(100)])

            models_.append(clf)

            if task_type == 'cls':
                val_preds = clf.predict_proba(X_val)[:,1]
                val_score = log_loss(y_val, val_preds)
            else:
                val_preds = clf.predict(X_val)
                val_score = mean_squared_error(y_val, val_preds, squared=False)
            
            oof[val_idx] = val_preds
            best_iter = clf.best_iteration_

            print(f'Fold: {blu}{fold + 1:>3}{res}| loss: {blu}{val_score:.5f}{res}| Best iteration: {blu}{best_iter:>4}{res}')

            # permutation importance
            if permut:
                perm = PermutationImportance(clf, scoring=None, n_iter=1, 
                                             random_state=42, cv=None, refit=False).fit(X_val, y_val)

                perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                                    index=X_val.columns).sort_index()

                if perm_df_.shape[0] == 0:
                    perm_df_ = perm_importance_df.copy()
                else:
                    perm_df_ += perm_importance_df

            # gboost feature importance
            f_i = pd.DataFrame(sorted(zip(clf.feature_importances_, X.columns), 
                                      reverse=True, key=lambda x: x[1]), 
                                columns=['Value','Feature'])

            if feature_importances_.shape[0] == 0:
                feature_importances_ = f_i.copy()
            else:
                feature_importances_['Value'] += f_i['Value']
                    
            # BORUTA importance
            if boruta:
                model = BoostBoruta(clf, importance_type='shap_importances', train_importance=False)
                try:
                    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                            eval_metric=eval_metric, 
                            callbacks=[lgb.log_evaluation(100)])
                except RuntimeError:
                    continue
                
                boruta_importance_df = pd.DataFrame({'importance': model.ranking_}, 
                                                        index=X_train.columns).sort_index()
                if boruta_df_.shape[0] == 0:
                    boruta_df_ = boruta_importance_df.copy()
                else:
                    boruta_df_ += boruta_importance_df

        if task_type == 'cls':
            outer_cv = log_loss(y, oof)
        else:
            outer_cv = mean_squared_error(y, oof, squared=False)
        
        outer_cv_score.append(outer_cv)

    print(f'{red} Outer Holdout avg score: {res} log_loss: {red}{np.mean(outer_cv_score):.5f}{res}')
    print(f'{"*" * 50}\n')
    
    if permut:
        perm_df_ = perm_df_.sort_values('importance', ascending=False)
        perm_df_ = perm_df_.reset_index().rename({'index': 'Feature'}, axis=1)
        
    if boruta:
        boruta_df_ = boruta_df_.sort_values('importance')
        boruta_df_ = boruta_df_.reset_index().rename({'index': 'Feature'}, axis=1)
                                    
    feature_importances_ = feature_importances_.sort_values('Value', ascending=False).reset_index(drop=True)
    
    return perm_df_, feature_importances_, boruta_df_, np.mean(outer_cv_score)


params = {
          'n_estimators': 1000,
          'learning_rate': 0.01,
        #   'early_stopping_round': 100,






        
          'max_depth': 7,
          'subsample' : 0.8,
          'colsample_bytree': 0.75,
          'num_leaves': 32,
          'verbosity': -1,
          'importance_type': 'gain'
        }

task_type = 'cls'

if task_type == 'cls':
    params['boosting_type'] = 'dart'
    params['objective'] = 'binary'
else:
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'

if CFG.select_features:
    perm_df_, feature_importances_, boruta_df_, outer_cv_score = lgbm_tuning(train_df, permut=True, boruta=True)

Repeat #1


Early stopping is not available in dart mode


[100]	training's binary_logloss: 0.533675	valid_1's binary_logloss: 0.551274
[200]	training's binary_logloss: 0.5017	valid_1's binary_logloss: 0.526743
[300]	training's binary_logloss: 0.463149	valid_1's binary_logloss: 0.498259
[400]	training's binary_logloss: 0.432549	valid_1's binary_logloss: 0.477228
[500]	training's binary_logloss: 0.40653	valid_1's binary_logloss: 0.460971
[600]	training's binary_logloss: 0.397043	valid_1's binary_logloss: 0.455439
[700]	training's binary_logloss: 0.382004	valid_1's binary_logloss: 0.447044
[800]	training's binary_logloss: 0.373078	valid_1's binary_logloss: 0.442644
[900]	training's binary_logloss: 0.361244	valid_1's binary_logloss: 0.437967
[1000]	training's binary_logloss: 0.353137	valid_1's binary_logloss: 0.434973
Fold:   1| loss: 0.43497| Best iteration:    0


Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray

[100]	training's binary_logloss: 0.527683	valid_1's binary_logloss: 0.575327
[200]	training's binary_logloss: 0.496245	valid_1's binary_logloss: 0.555903
[300]	training's binary_logloss: 0.457208	valid_1's binary_logloss: 0.532656
[400]	training's binary_logloss: 0.425823	valid_1's binary_logloss: 0.515708
[500]	training's binary_logloss: 0.399018	valid_1's binary_logloss: 0.501342
[600]	training's binary_logloss: 0.388914	valid_1's binary_logloss: 0.496184
[700]	training's binary_logloss: 0.373824	valid_1's binary_logloss: 0.488838
[800]	training's binary_logloss: 0.364619	valid_1's binary_logloss: 0.484329
[900]	training's binary_logloss: 0.352683	valid_1's binary_logloss: 0.480066
[1000]	training's binary_logloss: 0.34488	valid_1's binary_logloss: 0.477952
Fold:   2| loss: 0.47795| Best iteration:    0


...

[100]	training's binary_logloss: 0.537495	valid_1's binary_logloss: 0.538863
[200]	training's binary_logloss: 0.504867	valid_1's binary_logloss: 0.513183
[300]	training's binary_logloss: 0.466759	valid_1's binary_logloss: 0.482057
[400]	training's binary_logloss: 0.436811	valid_1's binary_logloss: 0.459332
[500]	training's binary_logloss: 0.410418	valid_1's binary_logloss: 0.441291
[600]	training's binary_logloss: 0.400525	valid_1's binary_logloss: 0.435264
[700]	training's binary_logloss: 0.384711	valid_1's binary_logloss: 0.425932
[800]	training's binary_logloss: 0.375551	valid_1's binary_logloss: 0.420991
[900]	training's binary_logloss: 0.364509	valid_1's binary_logloss: 0.414864
[1000]	training's binary_logloss: 0.356759	valid_1's binary_logloss: 0.411636
Fold:   4| loss: 0.41164| Best iteration:    0


Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray

[100]	training's binary_logloss: 0.536536	valid_1's binary_logloss: 0.540979
[200]	training's binary_logloss: 0.504673	valid_1's binary_logloss: 0.514187
[300]	training's binary_logloss: 0.466674	valid_1's binary_logloss: 0.483604
[400]	training's binary_logloss: 0.437056	valid_1's binary_logloss: 0.460467
[500]	training's binary_logloss: 0.411289	valid_1's binary_logloss: 0.442784
[600]	training's binary_logloss: 0.401685	valid_1's binary_logloss: 0.436312
[700]	training's binary_logloss: 0.387098	valid_1's binary_logloss: 0.426986
[800]	training's binary_logloss: 0.377713	valid_1's binary_logloss: 0.422134
[900]	training's binary_logloss: 0.366063	valid_1's binary_logloss: 0.416405
[1000]	training's binary_logloss: 0.357594	valid_1's binary_logloss: 0.413312
Fold:   5| loss: 0.41331| Best iteration:    0


Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
Early stopping is not available in dart mode
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray

 Outer Holdout avg score:  log_loss: 0.43357
**************************************************



# Combine importances and save them

In [11]:
if CFG.select_features:
    perm_df_['rank'] = perm_df_['importance'].rank(ascending=False)
    boruta_df_['rank'] = boruta_df_['importance'].rank()
    feature_importances_['rank'] = feature_importances_['Value'].rank(ascending=False)

    res = pd.concat([perm_df_[['Feature','rank']], boruta_df_[['Feature','rank']], feature_importances_[['Feature','rank']]])
    res = res.groupby('Feature')['rank'].sum().sort_values()
    res.to_csv('feature_importance.csv')
else:
    res = pd.read_csv('feature_importance.csv')

# Train model with selected features

In [39]:
def model_train(df, features, task_type, how, n_folds, low_bound, high_bound): 
    oof = np.zeros([df['target'].shape[0], 1])

    X, groups = df[features], df['ticker']
    X = pd.concat([X, pd.get_dummies(df['pattern'], drop_first=True)], axis=1)
    # X = pd.concat([X, pd.get_dummies(df['ttype'], drop_first=True)], axis=1)
    
    if task_type == 'cls':
        y = df['target']
        kf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=180820231)
    else:
        y = (df['target'] - df['close']) / df['close']
        kf = GroupKFold(n_splits=n_folds)

    oe_enc = OrdinalEncoder()
    groups = oe_enc.fit_transform(groups.values.reshape(-1, 1))

    print(f"Training with {len(features)} features")
    
    if how == 'lreg':
        scaler = StandardScaler()
        X[X.columns] = scaler.fit_transform(X)
    
    for fold, (fit_idx, val_idx) in enumerate(kf.split(X, y, groups)):
        print(f'Fold #{fold + 1}')
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]
        
        models = list()
        if how == 'lgbm':
            if task_type == 'cls':
                model = lgb.LGBMClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                        eval_metric='logloss', callbacks = [lgb.log_evaluation(100)])
                # best_iter = model.best_iteration_
            else:
                model = lgb.LGBMRegressor(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                        eval_metric='mse', callbacks = [lgb.log_evaluation(100)])
                # best_iter = model.best_iteration_
        elif how == 'lreg':
            if task_type == 'cls':
                model = LogisticRegression(C=0.1, max_iter=100000)#, class_weight='balanced')
                model.fit(X_train, y_train)
            else:
                model = LinearRegression(positive=True)
                model.fit(X_train, y_train)

        if task_type == 'cls':
            val_preds = model.predict_proba(X_val)
            val_score = log_loss(y_val, val_preds)
            prec_score, prec_obj_pct = confident_score(y_val, val_preds[:,1], low_bound, high_bound)
            print(f'Logloss: {val_score}, Confident objects precision: {prec_score}, % of confident objects: {prec_obj_pct}')
            oof[val_idx, 0] = val_preds[:,1]
        else:
            val_preds = model.predict(X_val)
            val_score = mean_squared_error(y_val, val_preds, squared=False)
            print('RMSE: {val_score}')
            oof[val_idx, 0] = val_preds
        
        models.append(model)
        
    return oof, models

def confident_score(y, oof, low_bound, high_bound):
    ''' Consider only high confident objects for accuracy and precision scores calculation;
        object probability must be lower than low_bound or higher than high_bound '''
    pred_conf = np.zeros_like(oof)
    pred_conf[oof > high_bound] = 1
    pred_conf[oof < low_bound] = 0
    pred_conf_acc = pred_conf[(oof < low_bound) | (oof > high_bound)]
    pred_conf_prec = pred_conf[(oof > high_bound)]

    y_conf_acc = y.values.reshape(-1,1)[(oof < low_bound) | (oof > high_bound)]
    y_conf_prec = y.values.reshape(-1,1)[(oof > high_bound)]

    return precision_score(y_conf_prec, pred_conf_prec), y_conf_prec.shape[0]/y.shape[0]

# task_type = 'reg'
task_type = 'cls'

# best params for classification
if task_type == 'cls':
    params = {
        'boosting_type': 'dart',
        'n_estimators': 1900,
        'learning_rate': 0.02,
        #   'early_stopping_round': 50,
        'max_depth': 10,
        'colsample_bytree': 0.65,
        'subsample': 0.8,
        'subsample_freq': 1,
        'num_leaves': 24,
        'verbosity': -1,
        'max_bin': 255,
        'reg_alpha': 1e-6,
        'reg_lambda': 1e-8,
        'objective': 'binary',
        # 'is_unbalance': True,
        # 'class_weight': 'balanced',
        'metric': 'average_precision'
        }
else:
    # best params for regression
    params = {
            'boosting_type': 'gbdt',
            'n_estimators': 1000,
            'learning_rate': 0.021,
            'early_stopping_round': 100,
            'max_depth': 7,
            'colsample_bytree': 0.75,
            'subsample': 0.8,
            'subsample_freq': 1,
            'num_leaves': 27,
            'verbosity': -1,
            'max_bin': 511,
            'reg_alpha': 1e-4,
            'reg_lambda': 1e-4,
            'objective': 'regression'
            }

if CFG.train_LGBM:
    low_bound, high_bound = 0.38, 0.62
    features = res.head(150).index.to_list()
    oof, models = model_train(train_df, features, task_type=task_type, how='lgbm', n_folds=5, low_bound=low_bound, high_bound=high_bound)

    if task_type == 'cls':
        y = train_df['target']
        oof_val_score = log_loss(y, oof)
        oof_conf_prec_score, oof_conf_obj_pct = confident_score(y, oof, low_bound, high_bound)
        print(f'Total Logloss: {oof_val_score}, Total confident objects precision: {oof_conf_prec_score}, Total % of confident objects: {oof_conf_obj_pct}')
    else:
        y = (train_df['target'] - train_df['close']) / train_df['close']
        display(mean_squared_error(y, oof, squared=False))


Training with 150 features
Fold #1
[100]	valid_0's binary_logloss: 0.67991	valid_0's average_precision: 0.62238
[200]	valid_0's binary_logloss: 0.67665	valid_0's average_precision: 0.623962
[300]	valid_0's binary_logloss: 0.673577	valid_0's average_precision: 0.625737
[400]	valid_0's binary_logloss: 0.671554	valid_0's average_precision: 0.628079
[500]	valid_0's binary_logloss: 0.671037	valid_0's average_precision: 0.627665
[600]	valid_0's binary_logloss: 0.670899	valid_0's average_precision: 0.627498
[700]	valid_0's binary_logloss: 0.671233	valid_0's average_precision: 0.622484
[800]	valid_0's binary_logloss: 0.670949	valid_0's average_precision: 0.62298
[900]	valid_0's binary_logloss: 0.669803	valid_0's average_precision: 0.625105
[1000]	valid_0's binary_logloss: 0.669142	valid_0's average_precision: 0.625675
[1100]	valid_0's binary_logloss: 0.668517	valid_0's average_precision: 0.62852
[1200]	valid_0's binary_logloss: 0.66745	valid_0's average_precision: 0.632921
[1300]	valid_0's bin

Total Logloss: 0.6582101740689003, Total confident objects precision: 0.7227138643067846, Total % of confident objects: 0.17532971295577968

In [ ]:
bad_features = list()
low_bound, high_bound = 0.4, 0.6
features = res.groupby('Feature')['rank'].sum().sort_values().head(150).index.to_list()
oof, models = model_train(train_df, features, task_type=task_type, how='lgbm', n_folds=5, low_bound=low_bound, high_bound=high_bound)
baseline_prec, baseline_pct = confident_score(y, oof, low_bound, high_bound)
print(f'Total confident objects precision: {baseline_prec}')

for feat in tqdm(features[30:]):
    tmp_features = [f for f in features if f != feat]
    oof, models = model_train(train_df, features, task_type=task_type, how='lgbm', n_folds=5, low_bound=low_bound, high_bound=high_bound)
    prec_score, pct = confident_score(y, oof, low_bound, high_bound)
    print(f'Feature: {feat}, Total precision: {prec_score}, Baseline precision: {baseline_prec}, Total pct: {pct}, Baseline precision: {baseline_pct}')
    if prec_score > baseline_prec and pct > baseline_pct:
        
        bad_features.append(feat)
        print(bad_features)

bad_features

In [ ]:
bad_features

['volume_prev_8']

150: Total Logloss: 0.6079201456679827, Total confident objects precision: 0.7093922651933702, Total % of confident objects: 0.07814523788964683